$\newcommand{\D}[2]{\frac{\partial #1}{\partial #2}}$ $\newcommand{\d}[2]{\partial #1/\partial #2}$
$\newcommand{\DD}[2]{\frac{\partial^2 #1}{\partial {#2}^2}}$ 
$\newcommand{\dd}[2]{\partial^2 #1/\partial {#2}^2}$

# GLM


Let $L(\beta)$ be the log-likelihood of the data $X, y$ as a function of the parameters $\beta$. 

The score (gradient) is the vector 
$$s = \D{L}{\beta}$$ 
with elements $s_i = \partial L/\partial \beta_i$. 


We assume that the likelihood is a function of an observation vector $y$ and a mean which is a function of the parameters, $\mu(\beta)$, thus

$$
L(\beta) = L(y, \mu(\beta))
$$

Then, using the chain rule, 

$$
s = \D{L}{\beta} = \D{L}{\mu}\D{\mu}{\beta}
$$


The term $\d{\mu}{\beta}$ is the jacobian $J$ of the predictor $\mu(\beta)$ with rows equal to the number of 
observations $n$ and columns equal to the number of parameters $b$. 
The term $\d{L}{\mu}$ is the gradient of the log-likelihood with respect to the expected value, and has rows equal to the
number of observations. This seems to be a normalized deviate. Thus

$$
s = \D{L}{\mu} J
$$

The einsum signature is `i,ij->j` where $\d{L}{\mu}$ and $\mu$ is indexed by i and $J$ by i & j ($\beta$ is indexed by j). 


The hessian is

$$
H = \DD{L}{\beta}=\DD{L}{\mu}\DD{\mu}{\beta}
$$

The expected value of this is

$$
E(H) = E\left(\DD{L}{\beta}\right) = E\left(\D{L}{\beta}\D{L}{\beta^*} \right)
$$

(which needs proving) in which $\beta$ and $\beta^*$ are the same vector, iterated over differently. We then have

$$
E\left(\D{L}{\beta}\D{L}{\beta^*} \right) = 
E\left(\D{L}{\mu}\D{\mu}{\beta}\D{L}{\mu^*}\D{\mu^*}{\beta^*}\right) =
E\left(\D{L}{\mu}\D{L}{\mu^*}\right)\D{\mu}{\beta}\D{\mu^*}{\beta^*}
$$

The term in the expectation is an outer product since it's over every combination of elements from $\mu$ and $\mu^*$. 

The only nonzero elements of $E\left(\d{L}{\mu}\,\d{L}{\mu*}\right)$ are when $\mu=\mu^*$, because the observations are independent, so this is a diagonal matrix, call it $V$, with elements

$$
V_{i,i} = E\left(\D{L_i}{\mu_i}\right)^2
$$

Thus

$$
E(H) = J'VJ
$$

In [1]:
import os
import sys
up = os.path.normpath(os.path.join(os.getcwd(), "../src"))
sys.path.append(up)
sys.path.append(os.getcwd())

## Reverse Approach.

This is trickier because the jacobian of the mu function must be computed using forward differentiation, for efficiency, and unfortunately that doesn't happen because the jacobian of the mu function has been traced & backtraced. This happens because the derivatives of components of a function are computed prior to the function itself, due to the pullback algorithm:

```
derivs = []  # holds dY/dZi
if True:  # id(n) not in deriv_memo:
    for Zi, argno in A.root.backrefs[A]:
        derivs.append(
            pullback(Zi, argno)
        )  # this is dY/dZi*dZi/dA,argno
    derivs = summate(derivs)
return derivs
```

So the predictor has to be packaged into the error model to give a function of data & beta.

In [2]:
from maxmf.autodiff.tracing import notrace, value_of
import maxmf.autodiff.forward as fwd
import maxmf.autodiff.reverse as rev

def GLM_Builder(score, dLdmu, varfunc):
    # creates a GLM likelihood function; this uses the -Fisher instead of the hessian.
    # dLdmu is a vector of the derivatives of the likelihood with respect to the expected value
    # varfunc is a vector of the expected value of dLdmu**2
    
    def GLM(meanfunc):
        # factory function to create likelihoods with derivatives
        # y is the data and meanfunc is a callable which takes (X,beta)
        # returns a function L(y,X,beta) which is notraced
        
        @notrace
        def L(y, X, b):
            return score(y, meanfunc(X,b))

        # reverse mode
        
        @rev.register_deriv(L)
        def dLrev(dYdZ, argno, y, X, b):
            # function to work out the meanfunc & jacobian then pass onto
            # the actual derivative. This step is necessary to memoize the
            # value of mu and J
            if argno!=2:
                return 0
            mfunc = fwd.Diff(meanfunc, argno=1)
            mu = mfunc.trace(X, value_of(b))
            J = mfunc.jacobian(gc=True)
            # we have to keep b as a parameter here so that it will be traced when
            # computing higher derivatives.
            # NB dydz is guaranteed to be a scalar
            return dYdZ*_dLrev(y, mu, J, b)        
        
        @notrace
        def _dLrev(y, mu, J, b):
            # works out the derivative of L given mu=meanfunc(X,b) & its jacobian
            return np.einsum('i,i...->...', dLdmu(y,mu), J)

        @rev.register_deriv(_dLrev)
        def dL2rev(dYdZ, argno, y, mu, J, b):
            # this doesn't depend on b so has no higher derivatives
            # works out the hessian of L given y, mu=meanfunc(X,b) and its jacobian
            if argno!=3:
                return 0
            # we aren't going to do any higher derivatives, so just return
            # a matrix
            # if J has more than 2 dim, we need to allow for this.
            return -dYdZ@np.einsum('i,ij,ik->jk',varfunc(y,mu),J,J)

        # forward mode
        
        @fwd.register_deriv(L)
        def dLfwd(y, X, b):
            mfunc = fwd.Diff(meanfunc, argno=1)
            mu = mfunc.trace(X, value_of(b))
            J = mfunc.jacobian(gc=True)
            return _dLfwd(y, mu, J, b)

        @notrace
        def _dLfwd(y, mu, J, b):
            # works out the derivative of L given mu=meanfunc(X,b) & its jacobian
            return np.einsum('i,i...->...', dLdmu(y,mu), J)
        
        @fwd.register_deriv(_dLfwd)
        def dL2fwd(y, mu, J, b):
            # this doesn't depend on b so has no higher derivatives
            return -np.einsum('i,ij,ik->jk',varfunc(y,mu),J,J)

        return L
    
    return GLM

## Gaussian Errors.

If the observation $y$ is gaussian with mean $\mu$, the likelihood is

$$
Pr(y|\mu) = C exp\left(-\frac{(y-\mu)^2}{2\sigma^2}\right)
$$

The log likelihood is

$$L = -\frac{(y-\mu)^2}{2\sigma^2} + \log{C}$$

Then

$$
\D{L}{\mu} = \frac{y-\mu}{\sigma^2}
$$

And

$$
E\left(\D{L}{\mu}\right)^2 = \frac{E(y-\mu)^2}{\sigma^4}=\frac{1}{\sigma^2}
$$

In this case, because the log-likelihood, the score and $E(H)$ are all multiplied by the same factor $1/\sigma^2$, we can ignore it, and have

$$
\begin{align}
L &= -0.5(y-\mu)^2 \\
\D{L}{\mu} &= y-\mu \\
E\left(\D{L}{\mu}\right)^2 &= 1
\end{align}
$$



In [3]:
Gaussian = GLM_Builder(lambda y,mu: -0.5*np.sum((y-mu)**2), 
                       lambda y,mu: y-mu, 
                       lambda y,mu: np.ones(y.shape))
Normal = Gaussian # synonym

In [4]:
import numpy as np

y = np.random.rand(10)
X = np.random.rand(10,3)
b = np.random.rand(3)

g = Gaussian(lambda X,b:X@b) # g is g(y,X,b)
func = lambda y,X,b: g(y,X,b)-0*np.sum(b**2)

func2 = fwd.Diff(func, argno=2)
f = func2.trace(y, X, b)
j = func2.jacobian()
print(j)
h = func2.hessian()
print(h)

func2 = rev.Diff(func, argno=2)
f = func2.trace(y, X, b)
j = func2.jacobian()
print(j)
h = func2.hessian(gc=False)
print(h)

[0.25139788 0.74841929 0.16438616]
[[-3.36786391 -2.79262764 -2.97563054]
 [-2.79262764 -4.17533082 -3.16090872]
 [-2.97563054 -3.16090872 -3.75644909]]
[0.25139788 0.74841929 0.16438616]
[[-3.36786391 -2.79262764 -2.97563054]
 [-2.79262764 -4.17533082 -3.16090872]
 [-2.97563054 -3.16090872 -3.75644909]]


In [5]:
def linreg(y,X,b):
    return -0.5*np.sum((y-X@b)**2)

func2 = fwd.Diff(linreg, argno=2)
f = func2.trace(y, X, b)
j = func2.jacobian()
print(j)
h = func2.hessian()
print(h)

[0.25139788 0.74841929 0.16438616]
[[-3.36786391 -2.79262764 -2.97563054]
 [-2.79262764 -4.17533082 -3.16090872]
 [-2.97563054 -3.16090872 -3.75644909]]


In [6]:
np.linalg.lstsq(h, -j , rcond=None)[0]+b

array([0.15127874, 0.62942399, 0.06286967])

## Binomial Errors.

If $y$ is a 0,1 observation with expected value $\mu$, then

$$
L = y\log(\mu) + (1-y)\log(1-\mu)
$$

The derivative of this wrt $\mu$ is

$$
\D{L}{\mu} = \frac{y}{\mu}-\frac{1-y}{1-\mu} = \frac{y(1-\mu)-(1-y)\mu}{\mu(1-\mu)}=\frac{y-\mu}{\mu(1-\mu)}
$$

Finally, 

$$
E\left(\D{L}{\mu}\right)^2 = \frac{E(y-\mu)^2}{(\mu(1-\mu))^2}=\frac{1}{\mu(1-\mu)}
$$

In [7]:

clip = [0.001, 0.999]

def logl(y,mu):
    cmu = np.clip(mu, *clip)
    return np.sum(y*np.log(cmu)+(1-y)*np.log(1-cmu))

def dLdmu(y,mu):
    cmu = np.clip(mu, *clip)
    unclipped = cmu==mu
    return unclipped*(y-cmu)/(cmu*(1-cmu))

def varfunc(y,mu):
    cmu = np.clip(mu, *clip)
    unclipped = cmu==mu
    return unclipped/(mu*(1-mu))

Binomial = GLM_Builder(logl, dLdmu, varfunc )

## Poisson Errors.

$$
L \propto y_i\log{\mu_i} -\mu_i
$$

$$
\D{L}{\mu} = \frac{y}{\mu}-1 = \frac{y-\mu}{\mu}
$$

$$
E\left( \D{L}{\mu} \right)^2 = \frac{E((y-\mu)^2)}{\mu^2} = \frac{\mu}{\mu^2} = \frac{1}{\mu}
$$

and $\mu>0$

In [8]:
def logl(y,mu):
    unclipped = mu>0
    cmu = mu*unclipped
    return np.sum(y*np.log(cmu)-cmu)
    
def dLdmu(y,mu):
    unclipped = mu>0
    return unclipped*(y-mu)/(1e-10+mu*unclipped)

def varfunc(y,mu):
    unclipped = mu>0
    return unclipped/(1e-10+mu*unclipped)

Poisson = GLM_Builder(logl, dLdmu, varfunc)

## Negative Binomial.

The probability of the number of failures before $1/\alpha$ successes have occurred.

Parameters are $\alpha>0$ and $\mu$, such that $\alpha\mu$ is positive.

The likelihood is

$$
Pr(y|\mu) = C \left(\frac{1}{1+\alpha\mu}\right)^{\frac{1}{\alpha}}
       \left(\frac{\alpha\mu}{1+\alpha\mu}\right)^y
$$

where $C$ is a binomial coefficient. This is the parameterization chosen by statsmodels, because $E(y)=\mu$.

The log-likelihood is thus

$$
L(y, \mu) = \log{C} + \frac{1}{\alpha}\log\left(\frac{1}{1+\alpha\mu}\right)+y\log\left(\frac{\alpha\mu}{1+\alpha\mu}\right) 
$$

Thus

$$
\D{L}{\mu} = \frac{y}{\mu}-\frac{1+\alpha y}{1+\alpha\mu}=\frac{y-\mu}{\mu+\alpha\mu^2}
$$

and

$$
E\left(\D{L}{\mu}\right)^2 = \frac{1}{\mu+\alpha\mu^2}
$$

In [9]:
# factory to make negative binomial distributions:

def NegativeBinomial(alpha):
    # use as NegativeBinomial(alpha)(predictor)
    
    def logl(y,mu):
        unclipped = mu>0
        cmu = mu*unclipped
        amu = alpha*cmu
        return np.sum(-(1/alpha)*np.log(1+amu)+y*np.log(amu/(1+amu)))

    def dLdmu(y,mu):
        unclipped = mu>0
        cmu = unclipped*mu
        return unclipped*(y-mu)/(1e-10+cmu+alpha*cmu)

    def varfunc(y,mu):
        unclipped = mu>0
        cmu = unclipped*mu
        return 1.0/(1e-10+cmu+alpha*cmu)

    return GLM_Builder(logl, dLdmu, varfunc)

gamma, multinomial to do.

## Mean Functions.

These functions give the derivatives used in GLMs. In GLM they are called mean functions.

In [10]:
@notrace
def LinearMF(X,b):
    # used with Gaussian
    return X@b

@fwd.register_deriv(LinearMF)
def dLinearMF(X,b):
    return X

LinearGLM = Gaussian(LinearMF)

@notrace
def LogitMF(X,b):
    # used with Binomial
    nu = X@b
    return 1/(1+np.exp(-nu))

@fwd.register_deriv(LogitMF)
def dLogitMF(X,b):
    exp_nu = np.exp(-X@b)
    return np.einsum('i,ij->ij', exp_nu/(1+exp_nu)**2, X)

LogisticGLM = Binomial(LogitMF)
    
@notrace
def ExpMF(X,b):
    # used with Poisson
    return np.exp(X@b)

@fwd.register_deriv(ExpMF)
def dExpMF(X,b):
    return np.einsum('i,ij->ij', np.exp(X@b), X)

PoissonGLM = Poisson(ExpMF)

In [11]:
y = np.random.rand(10)
X = np.random.rand(10,3)
b = 0*np.random.rand(3)

func = lambda b: LinearGLM(y,X,b)-0*np.sum(b**2)

func2 = fwd.Diff(func)
f = func2.trace(b)
j = func2.jacobian()
print(j)
h = func2.hessian()
print(h)

func2 = rev.Diff(func)
f = func2.trace(b)
j = func2.jacobian()
print(j)
h = func2.hessian(gc=False)
print(h)

[1.7983891  2.03881795 2.95421102]
[[-2.0250884  -1.51902008 -2.52993992]
 [-1.51902008 -3.07074464 -2.41104157]
 [-2.52993992 -2.41104157 -4.42575487]]
[1.7983891  2.03881795 2.95421102]
[[-2.0250884  -1.51902008 -2.52993992]
 [-1.51902008 -3.07074464 -2.41104157]
 [-2.52993992 -2.41104157 -4.42575487]]


In [12]:
X.T@X

array([[2.0250884 , 1.51902008, 2.52993992],
       [1.51902008, 3.07074464, 2.41104157],
       [2.52993992, 2.41104157, 4.42575487]])

In [13]:
X.T@y

array([1.7983891 , 2.03881795, 2.95421102])